# A Customer Support Q&A Chatbot

Traditionally, chatbots were built for specific user intents, formed from sets of sample questions and their corresponding answers. For example, a “Restaurant Recommendations” intent might include questions like “Can you suggest a good Italian restaurant nearby?” or “Where is the best sushi in town?” along with answers such as “La Trattoria is a great Italian restaurant in the area” or “Sushi Palace is highly recommended for sushi.”

In this framework, the chatbot matches user queries to the closest intent to generate a response. However, with the advancement of LLMs, the approach to developing chatbots is also evolving. Modern chatbots are increasingly sophisticated, offering more dynamic and nuanced responses to a broader array of user questions.

Large language models (LLMs) can significantly enhance chatbot functionality by linking broader intents with documents from a knowledge base (KB). This approach simplifies the handling of intents and enables more tailored responses to user queries.

However, LLMs have a maximum context size. For example, GPT-3 had a maximum prompt size limit of approximately 4,000 tokens. While this is substantial, when including an entire knowledge base in a single prompt it proves insufficient. Future advancements may overcome this limitation, and the current best LLMs have a bigger context size than GPT-3 (e.g., GPT-4 had an 8,000 token limit, and GPT-4o 128,000). In the meantime, it is crucial to devise solutions that work within the current constraints. Plus, more input tokens do not mean the right information is retrieved. It may add more noise than good information and contribute to hallucinations. It is thus necessary to curate and add relevant information to the input.

In the following tutorial, we will create a customer service chatbot, leveraging LLMs and text splitters so that the contextual information added to the prompt is within the context size constraints of the specific LLM.

## Workflow

This project aims to build a chatbot that leverages GPT-3 to search for answers within documents. The documents in this tutorial will be general technical guides scraped from the web.

The workflow for the project is explained below:

![image](./customer_support_chatbot_pipeline.jpg)

*Our customer support Q&A Chatbot pipeline.*

The first step is extracting content from internet webpages, dividing it into small parts, computing its embeddings, and storing it in Deep Lake. Subsequently, a user’s query retrieves the most relevant segments from Deep Lake. These segments are then incorporated into a prompt to generate the final response by the LLM.

To begin managing conversations with GPT-3 and storing data in Deep Lake, configure the OPENAI_API_KEY and ACTIVELOOP_TOKEN environment variables with your respective API keys and tokens.

We will use the `SeleniumURLLoader` class from the LangChain toolkit, which relies on the unstructured and selenium Python libraries. These can be installed via pip. Installing the latest version of these libraries is advisable, but this code has been explicitly tested with version 0.7.7.

In [ ]:
import os
from langchain_custom_utils.helper import get_openai_api_key, get_deeplake_api_key, print_response
OPENAI_API_KEY = get_openai_api_key()
DEEPLAKE_API_KEY = get_deeplake_api_key()

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain.document_loaders import SeleniumURLLoader
from langchain import PromptTemplate

These libraries offer features essential for developing *a context-aware question-answering system*, including managing **OpenAI embeddings**, handling vector storage, segmenting text, and interfacing with the **OpenAI API**. They play a crucial role in creating a system that combines information retrieval and text generation.

For this example, our chatbot’s database will contain technical content.

In [ ]:
# we'll use information from the following articles
urls = ['https://beebom.com/what-is-nft-explained/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-download-gif-twitter/',
        'https://beebom.com/how-use-chatgpt-linux-terminal/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-save-instagram-story-with-music/',
        'https://beebom.com/how-install-pip-windows/',
        'https://beebom.com/how-check-disk-usage-linux/']

### Split the documents into chunks and compute their embeddings

Load the documents from the provided URLs and split them into chunks using the `CharacterTextSplitter` with a chunk size of 1000 and no overlap:

In [ ]:
# use the selenium scraper to load the documents
loader = SeleniumURLLoader(urls=urls)
docs_not_splitted = loader.load()

# we split the documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs_not_splitted)



Next, obtain the embeddings using **OpenAIEmbeddings** and store them in a cloud-based Deep Lake vector database. In a real-world project, one might upload a whole website or course to Deep Lake to search across thousands or millions of documents. Utilizing a cloud serverless Deep Lake dataset enables applications in various locations to access the same centralized dataset without deploying a vector store on a specific computer.

Change the code below to include your Activeloop organization ID. By default, your org id is your username.

In [ ]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = DEEPLAKE_API_KEY
my_activeloop_dataset_name = "langchain_course_customer_support"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

To retrieve the most similar chunks to a given query, we can use the `similarity_search` method of the Deep Lake vector database:

In [ ]:
# let's see the top relevant documents to a specific query
query = "how to check disk usage in linux?"
docs = db.similarity_search(query)
print(docs[0].page_content)

The previous code will show something like the following output:

    Home How To How to Check Disk Usage in Linux (4 Methods)  
      
    How to Check Disk Usage in Linux (4 Methods)  
      
    Beebom Staff  
      
    Last Updated: February 21, 2023 3:15 pm  
      
    There may be times when you need to download some important files or transfer some photos to your Linux system, but face a problem of insufficient disk space. You head over to your file manager to delete the large files which you no longer require, but you have no clue which of them are occupying most of your disk space. In this article, we will show some easy methods to check disk usage in Linux from both the terminal and the GUI application.  
      
    Monitor Disk Usage in Linux (2023)  
      
    Table of Contents  
      
    Check Disk Space Using the df Command  
      
    Display Disk Usage in Human Readable FormatDisplay Disk Occupancy of a Particular Type  
      
    Check Disk Usage using the du Command  
      
    Display Disk Usage in Human Readable FormatDisplay Disk Usage for a Particular DirectoryCompare Disk Usage of Two Directories

### Craft a prompt for using the suggested strategies

For this chatbot, we will develop a prompt template that includes role prompting, knowledge base information, and the user’s question:

In [ ]:
# let's write a prompt for a customer support chatbot that
# answer questions using information extracted from our db
template = """You are an exceptional customer support chatbot that gently answer questions.

You know the following context information.

{chunks_formatted}

Answer to the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {query}

Answer:"""

prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template,
)

The template positions the chatbot as an advanced customer support tool and requires input variables: `chunks_formatted`, consisting of pre-arranged segments from *articles*, and *query*, representing the customer’s inquiry. The objective is to generate a precise and factual answer based on the provided segments, ensuring the information is accurate and not fabricated.

### Generate answers with the LLM

To generate a response, we retrieve the top-k (e.g., top-3) chunks most similar to the user’s question, format the prompt, and send it to the model at 0 temperature.


In [ ]:
# the full pipeline

# user question
query = "How to check disk usage in linux?"

# retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)

# generate answer
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
answer = llm(prompt_formatted)
print(answer)

> You can check disk usage in Linux using the df command to check disk
> space and the du command to check disk usage. You can also use the GUI
> application to check disk usage in a human readable format. For more
> information, please refer to the article "How to Check Disk Usage in
> Linux (4 Methods)" on Beebom.


In the previous example, while the chatbot generally functions effectively, there are scenarios where it might not perform as expected.

For instance, when the LLM is asked, “Is the Linux distribution free?” and provided with a context document about kernel features, it may incorrectly respond with “Yes, the Linux distribution is free to download and use,” even if this information isn’t in the provided context. Generating incorrect information is a significant concern for customer service chatbots.

The likelihood of the LLM producing inaccurate information decreases when the context directly includes the answer to the user’s query. However, since user inquiries are often short and vague, there’s always a possibility that the semantic search phase doesn’t retrieve appropriate documents. Embeddings play an important role in making the semantic search phase work as intended.